In [1]:
param_x_api_key = '2BeBXg69osbuG45Rjtv1wLEnHL3nuT3WPQQK1EaYHT4PutPkDrXB'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

import math

# = = = = = = = = = = = = = = =

df = pd.read_excel('./target.xlsx',
                   header=0,
                   dtype=str).fillna('')

input_ = df.groupby(df['Article_Id'], as_index=False).agg({'AITI': ';'.join})

dict_part = {}
list_job = []
for i in range(len(input_)):
    list_aiti = input_.loc[i, 'AITI'].split(';')

    list_list_aiti = []
    for j in range(math.ceil(len(list_aiti) / 25)):
        list_list_aiti.append(list_aiti[j*25:j*25+25])

    dict_part[input_.loc[i, 'Article_Id']] = {'aiti': len(list_list_aiti),
                                              'crawler': 0,
                                              'dict_vehicle': {},
                                              'vehicle_1': '',
                                              'vehicle_2': ''}

    for j in list_list_aiti:
        list_job.append((i, j))
        
print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(list_job)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                i, j = list_job[a]

                data = '''{"getArticleLinkedAllLinkingTargetsByIds3":
                {"provider":40000,
                 "lang":"en",
                 "articleCountry":"DE",
                 "articleId":''' + input_.loc[i, 'Article_Id'] + ''',
                 "immediateAttributs":true,
                 "linkingTargetType":"VOLB",
                 "linkedArticlePairs":{"array":['''

                for k in range(len(j)):
                    data += '''{"articleLinkId":"''' + j[k].split('-')[0] + '''","linkingTargetId":"''' + j[k].split('-')[1] + '''"},'''

                data = data[:-1] + ''']}}}'''

                headers = UA.get_User_Agent_Requests()
                headers['X-Api-Key'] = param_x_api_key

                resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = json_data['data']['array']
                for vehicle in list_vehicle:
                    make = vehicle['linkedVehicles']['array'][0]['manuDesc'].strip()
                    model = vehicle['linkedVehicles']['array'][0]['modelDesc'].strip()

                    engine = str(vehicle['linkedVehicles']['array'][0]['cylinderCapacity']) + 'cc - ' if 'cylinderCapacity' in vehicle['linkedVehicles']['array'][0] else ''
                    if vehicle['linkedArticleImmediateAttributs'] == '':
                        if engine != '':
                            engine = engine[:-3]
                    else:
                        list_attr = vehicle['linkedArticleImmediateAttributs']['array']
                        for attr in list_attr:
                            engine += attr['attrName'].strip()
                            if 'attrValue' in attr:
                                engine += ': ' + attr['attrValue'].strip() + ';'
                            else:
                                engine = engine + ';'
                        engine = engine[:-1]

                    list_year = []
                    if 'yearOfConstructionFrom' in vehicle['linkedVehicles']['array'][0]:
                        list_year.append(str(vehicle['linkedVehicles']['array'][0]['yearOfConstructionFrom'])[:4])
                    if 'yearOfConstructionTo' in vehicle['linkedVehicles']['array'][0]:
                        list_year.append(str(vehicle['linkedVehicles']['array'][0]['yearOfConstructionTo'])[:4])

                    if make not in dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle']:
                        dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make] = {model: {engine: list_year}}
                    else:
                        if model not in dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make]:
                            dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model] = {engine: list_year}
                        else:
                            if engine not in dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model]:
                                dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model][engine] = list_year
                            else:
                                dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model][engine] += list_year

                    dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model][engine] = sorted(list(set(dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model][engine])))

                dict_part[input_.loc[i, 'Article_Id']]['crawler'] += 1

                # = = = = = = = = = = = = = = =
                
                if dict_part[input_.loc[i, 'Article_Id']]['aiti'] == dict_part[input_.loc[i, 'Article_Id']]['crawler']:
                    list_vehicle = []
                    for make in dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle']:
                        for model in dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make]:
                            list_year = []
                            for engine in dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model]:
                                list_year += dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model][engine]
                            list_year = sorted(list(set(list_year)))

                            year_min = min(list_year)
                            year_max = max(list_year)
                            if year_min == year_max:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    dict_part[input_.loc[i, 'Article_Id']]['vehicle_1'] = '\n'.join(sorted(list_vehicle))
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_vehicle = []
                    for make in dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle']:
                        for model in dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make]:
                            for engine in dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model]:
                                year_min = min(dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model][engine])
                                year_max = max(dict_part[input_.loc[i, 'Article_Id']]['dict_vehicle'][make][model][engine])
                                if year_min == year_max:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                                else:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                    dict_part[input_.loc[i, 'Article_Id']]['vehicle_2'] = '\n'.join(sorted(list_vehicle))
                    
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'Article_Id': input_.loc[i, 'Article_Id'],
                                             'Vehicle_1': dict_part[input_.loc[i, 'Article_Id']]['vehicle_1'],
                                             'Vehicle_2': dict_part[input_.loc[i, 'Article_Id']]['vehicle_2']}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Article_Id': input_.loc[i, 'Article_Id'],
                                     'AITI': input_.loc[i, 'AITI']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[i, 'Article_Id'] + ' - ' + str(dict_part[input_.loc[i, 'Article_Id']]['crawler']) + '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Article_Id'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = pd.merge(left=output_error[['Article_Id']],
                            right=df,
                            how='left',
                            left_on='Article_Id',
                            right_on='AITI')
    output_error = output_error.sort_values(by=['Article_Id'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1814

[ok] - 1095867956 - 1
[尝试次数：1] - [剩余数量：1799] - [当前时间：14:36:09]

[ok] - 1095902317 - 1
[尝试次数：1] - [剩余数量：1798] - [当前时间：14:36:09]

[ok] - 1095902317 - 2
[尝试次数：1] - [剩余数量：1797] - [当前时间：14:36:09]

[ok] - 1130438181 - 1
[尝试次数：1] - [剩余数量：1796] - [当前时间：14:36:10]

[ok] - 1130438181 - 2
[尝试次数：1] - [剩余数量：1795] - [当前时间：14:36:10]

[ok] - 1130438181 - 3
[尝试次数：1] - [剩余数量：1794] - [当前时间：14:36:10]

[ok] - 1095800452 - 1
[尝试次数：1] - [剩余数量：1793] - [当前时间：14:36:10]

[ok] - 1130438181 - 4
[尝试次数：1] - [剩余数量：1792] - [当前时间：14:36:10]

[ok] - 1130438181 - 5
[尝试次数：1] - [剩余数量：1791] - [当前时间：14:36:10]

[ok] - 1095800452 - 2
[尝试次数：1] - [剩余数量：1790] - [当前时间：14:36:10]

[ok] - 1130438181 - 6
[尝试次数：1] - [剩余数量：1789] - [当前时间：14:36:10]

[ok] - 1130438181 - 7
[尝试次数：1] - [剩余数量：1788] - [当前时间：14:36:10]

[ok] - 1130438181 - 8
[尝试次数：1] - [剩余数量：1787] - [当前时间：14:36:10]

[ok] - 1130452985 - 1
[尝试次数：1] - [剩余数量：1786] - [当前时间：14:36:10]

[ok] - 1152765695 - 1
[尝试次数：1] - [剩余数量：1785] - [当前时间：14:36:10]

[ok] - 1130438181 - 9
[尝试次数：1]

[ok] - 1280098722 - 33
[尝试次数：2] - [剩余数量：1670] - [当前时间：14:36:28]

[ok] - 1280098722 - 34
[尝试次数：2] - [剩余数量：1669] - [当前时间：14:36:28]

[ok] - 1280098722 - 35
[尝试次数：2] - [剩余数量：1668] - [当前时间：14:36:29]

[ok] - 1280098722 - 36
[尝试次数：3] - [剩余数量：1667] - [当前时间：14:36:29]

[ok] - 1280098722 - 37
[尝试次数：2] - [剩余数量：1666] - [当前时间：14:36:29]

[ok] - 1280098722 - 38
[尝试次数：3] - [剩余数量：1665] - [当前时间：14:36:30]

[ok] - 1280098722 - 39
[尝试次数：2] - [剩余数量：1664] - [当前时间：14:36:30]

[ok] - 1280098722 - 40
[尝试次数：1] - [剩余数量：1663] - [当前时间：14:36:30]

[ok] - 1280098722 - 41
[尝试次数：6] - [剩余数量：1662] - [当前时间：14:36:31]

[ok] - 1280098722 - 42
[尝试次数：4] - [剩余数量：1661] - [当前时间：14:36:31]

[ok] - 1280098722 - 43
[尝试次数：2] - [剩余数量：1660] - [当前时间：14:36:31]

[ok] - 1280098722 - 44
[尝试次数：2] - [剩余数量：1659] - [当前时间：14:36:31]

[ok] - 1280098722 - 45
[尝试次数：1] - [剩余数量：1658] - [当前时间：14:36:32]

[ok] - 1280098722 - 46
[尝试次数：5] - [剩余数量：1657] - [当前时间：14:36:32]

[ok] - 1280098722 - 47
[尝试次数：1] - [剩余数量：1656] - [当前时间：14:36:32]

[ok] - 1280098722 - 48
[尝

[ok] - 1362844266 - 4
[尝试次数：1] - [剩余数量：1543] - [当前时间：14:37:02]

[ok] - 1362844266 - 5
[尝试次数：1] - [剩余数量：1542] - [当前时间：14:37:02]

[ok] - 1362844266 - 6
[尝试次数：1] - [剩余数量：1541] - [当前时间：14:37:03]

[ok] - 1362844266 - 7
[尝试次数：1] - [剩余数量：1540] - [当前时间：14:37:03]

[ok] - 1362844266 - 8
[尝试次数：1] - [剩余数量：1539] - [当前时间：14:37:03]

[ok] - 1362844266 - 9
[尝试次数：1] - [剩余数量：1538] - [当前时间：14:37:04]

[ok] - 1362844266 - 10
[尝试次数：1] - [剩余数量：1537] - [当前时间：14:37:04]

[ok] - 1362844266 - 11
[尝试次数：1] - [剩余数量：1536] - [当前时间：14:37:04]

[ok] - 1373705771 - 1
[尝试次数：1] - [剩余数量：1535] - [当前时间：14:37:05]

[ok] - 1362844266 - 12
[尝试次数：1] - [剩余数量：1534] - [当前时间：14:37:05]

[ok] - 1363787144 - 1
[尝试次数：1] - [剩余数量：1533] - [当前时间：14:37:06]

[ok] - 1363422796 - 1
[尝试次数：1] - [剩余数量：1532] - [当前时间：14:37:06]

[ok] - 1373708339 - 1
[尝试次数：1] - [剩余数量：1531] - [当前时间：14:37:06]

[ok] - 1373706123 - 1
[尝试次数：1] - [剩余数量：1530] - [当前时间：14:37:06]

[ok] - 1373708371 - 1
[尝试次数：1] - [剩余数量：1529] - [当前时间：14:37:06]

[ok] - 1388919874 - 1
[尝试次数：1] - [剩余数

[ok] - 44289978 - 1
[尝试次数：1] - [剩余数量：1412] - [当前时间：14:38:05]

[ok] - 44289978 - 2
[尝试次数：1] - [剩余数量：1411] - [当前时间：14:38:05]

[ok] - 44289978 - 3
[尝试次数：1] - [剩余数量：1410] - [当前时间：14:38:06]

[ok] - 1280200921 - 7
[尝试次数：2] - [剩余数量：1409] - [当前时间：14:38:06]

[ok] - 44291943 - 1
[尝试次数：1] - [剩余数量：1408] - [当前时间：14:38:06]

[ok] - 44291943 - 2
[尝试次数：1] - [剩余数量：1407] - [当前时间：14:38:07]

[ok] - 1306816690 - 1
[尝试次数：2] - [剩余数量：1406] - [当前时间：14:38:07]

[ok] - 44314325 - 1
[尝试次数：1] - [剩余数量：1405] - [当前时间：14:38:07]

[ok] - 44314354 - 1
[尝试次数：1] - [剩余数量：1404] - [当前时间：14:38:08]

[ok] - 44340968 - 1
[尝试次数：1] - [剩余数量：1403] - [当前时间：14:38:08]

[ok] - 44291943 - 3
[尝试次数：1] - [剩余数量：1402] - [当前时间：14:38:08]

[ok] - 44340968 - 2
[尝试次数：1] - [剩余数量：1401] - [当前时间：14:38:08]

[ok] - 44173566 - 2
[尝试次数：1] - [剩余数量：1400] - [当前时间：14:38:08]

[ok] - 44340968 - 3
[尝试次数：1] - [剩余数量：1399] - [当前时间：14:38:09]

[ok] - 44314243 - 1
[尝试次数：1] - [剩余数量：1398] - [当前时间：14:38:09]

[ok] - 44340968 - 4
[尝试次数：1] - [剩余数量：1397] - [当前时间：14:38:09]

[ok]

[ok] - 44765004 - 2
[尝试次数：1] - [剩余数量：1280] - [当前时间：14:38:45]

[ok] - 44764956 - 4
[尝试次数：1] - [剩余数量：1279] - [当前时间：14:38:45]

[ok] - 44754842 - 9
[尝试次数：1] - [剩余数量：1278] - [当前时间：14:38:45]

[ok] - 44765004 - 3
[尝试次数：1] - [剩余数量：1277] - [当前时间：14:38:46]

[ok] - 44765004 - 4
[尝试次数：1] - [剩余数量：1276] - [当前时间：14:38:46]

[ok] - 44765004 - 5
[尝试次数：1] - [剩余数量：1275] - [当前时间：14:38:46]

[ok] - 44765004 - 6
[尝试次数：1] - [剩余数量：1274] - [当前时间：14:38:46]

[ok] - 44765004 - 7
[尝试次数：1] - [剩余数量：1273] - [当前时间：14:38:47]

[ok] - 44754842 - 10
[尝试次数：1] - [剩余数量：1272] - [当前时间：14:38:47]

[ok] - 44765004 - 8
[尝试次数：1] - [剩余数量：1271] - [当前时间：14:38:47]

[ok] - 44765004 - 9
[尝试次数：1] - [剩余数量：1270] - [当前时间：14:38:48]

[ok] - 44765004 - 10
[尝试次数：1] - [剩余数量：1269] - [当前时间：14:38:48]

[ok] - 44765004 - 11
[尝试次数：1] - [剩余数量：1268] - [当前时间：14:38:48]

[ok] - 44754842 - 11
[尝试次数：1] - [剩余数量：1267] - [当前时间：14:38:48]

[ok] - 44765004 - 12
[尝试次数：1] - [剩余数量：1266] - [当前时间：14:38:48]

[ok] - 44765004 - 13
[尝试次数：1] - [剩余数量：1265] - [当前时间：14:38:49]

[o

...

[ok] - 46661566 - 5
[尝试次数：2] - [剩余数量：614] - [当前时间：14:41:36]

[ok] - 46788811 - 19
[尝试次数：1] - [剩余数量：613] - [当前时间：14:41:36]

[ok] - 46837054 - 1
[尝试次数：1] - [剩余数量：612] - [当前时间：14:41:37]

[ok] - 46837054 - 2
[尝试次数：1] - [剩余数量：611] - [当前时间：14:41:37]

[ok] - 46837054 - 3
[尝试次数：1] - [剩余数量：610] - [当前时间：14:41:37]

[ok] - 46837054 - 4
[尝试次数：1] - [剩余数量：609] - [当前时间：14:41:38]

[ok] - 46837054 - 5
[尝试次数：1] - [剩余数量：608] - [当前时间：14:41:38]

[ok] - 46837359 - 1
[尝试次数：1] - [剩余数量：607] - [当前时间：14:41:39]

[ok] - 46362255 - 29
[尝试次数：2] - [剩余数量：606] - [当前时间：14:41:39]

[ok] - 46837054 - 6
[尝试次数：1] - [剩余数量：605] - [当前时间：14:41:39]

[ok] - 46837359 - 2
[尝试次数：1] - [剩余数量：604] - [当前时间：14:41:39]

[ok] - 46821227 - 2
[尝试次数：1] - [剩余数量：603] - [当前时间：14:41:40]

[ok] - 46837359 - 3
[尝试次数：1] - [剩余数量：602] - [当前时间：14:41:40]

[ok] - 46837359 - 4
[尝试次数：1] - [剩余数量：601] - [当前时间：14:41:40]

[ok] - 46948890 - 1
[尝试次数：1] - [剩余数量：600] - [当前时间：14:41:42]

[ok] - 46948890 - 2
[尝试次数：1] - [剩余数量：599] - [当前时间：14:41:42]

[ok] - 46837359 - 5
[尝

[ok] - 47328985 - 2
[尝试次数：1] - [剩余数量：478] - [当前时间：14:42:36]

[ok] - 46657588 - 6
[尝试次数：2] - [剩余数量：477] - [当前时间：14:42:37]

[ok] - 47350812 - 2
[尝试次数：1] - [剩余数量：476] - [当前时间：14:42:37]

[ok] - 47389179 - 1
[尝试次数：1] - [剩余数量：475] - [当前时间：14:42:37]

[ok] - 46657588 - 7
[尝试次数：2] - [剩余数量：474] - [当前时间：14:42:37]

[ok] - 47350812 - 3
[尝试次数：1] - [剩余数量：473] - [当前时间：14:42:38]

[ok] - 47401951 - 1
[尝试次数：1] - [剩余数量：472] - [当前时间：14:42:38]

[ok] - 47389179 - 2
[尝试次数：1] - [剩余数量：471] - [当前时间：14:42:38]

[ok] - 47401692 - 1
[尝试次数：1] - [剩余数量：470] - [当前时间：14:42:38]

[ok] - 47401952 - 1
[尝试次数：1] - [剩余数量：469] - [当前时间：14:42:38]

[ok] - 47328985 - 3
[尝试次数：1] - [剩余数量：468] - [当前时间：14:42:39]

[ok] - 47458102 - 1
[尝试次数：1] - [剩余数量：467] - [当前时间：14:42:39]

[ok] - 47410099 - 1
[尝试次数：1] - [剩余数量：466] - [当前时间：14:42:39]

[ok] - 47389179 - 3
[尝试次数：1] - [剩余数量：465] - [当前时间：14:42:39]

[ok] - 47458102 - 2
[尝试次数：1] - [剩余数量：464] - [当前时间：14:42:39]

[ok] - 47458102 - 3
[尝试次数：1] - [剩余数量：463] - [当前时间：14:42:40]

[ok] - 46661489 - 6
[尝试次

[ok] - 47946076 - 8
[尝试次数：1] - [剩余数量：343] - [当前时间：14:43:11]

[ok] - 47946076 - 9
[尝试次数：1] - [剩余数量：342] - [当前时间：14:43:11]

[ok] - 47188609 - 1
[尝试次数：2] - [剩余数量：341] - [当前时间：14:43:11]

[ok] - 47946076 - 10
[尝试次数：1] - [剩余数量：340] - [当前时间：14:43:12]

[ok] - 47946076 - 11
[尝试次数：1] - [剩余数量：339] - [当前时间：14:43:12]

[ok] - 47946076 - 12
[尝试次数：1] - [剩余数量：338] - [当前时间：14:43:13]

[ok] - 47946076 - 13
[尝试次数：1] - [剩余数量：337] - [当前时间：14:43:13]

[ok] - 47946076 - 14
[尝试次数：1] - [剩余数量：336] - [当前时间：14:43:13]

[ok] - 47946076 - 15
[尝试次数：1] - [剩余数量：335] - [当前时间：14:43:13]

[ok] - 47946076 - 16
[尝试次数：1] - [剩余数量：334] - [当前时间：14:43:14]

[ok] - 47946076 - 17
[尝试次数：1] - [剩余数量：333] - [当前时间：14:43:14]

[ok] - 47969642 - 1
[尝试次数：1] - [剩余数量：332] - [当前时间：14:43:14]

[ok] - 47969642 - 2
[尝试次数：1] - [剩余数量：331] - [当前时间：14:43:14]

[ok] - 47969642 - 3
[尝试次数：1] - [剩余数量：330] - [当前时间：14:43:14]

[ok] - 47983609 - 1
[尝试次数：1] - [剩余数量：329] - [当前时间：14:43:14]

[ok] - 47218374 - 5
[尝试次数：2] - [剩余数量：328] - [当前时间：14:43:15]

[ok] - 47977040 

[ok] - 48474546 - 4
[尝试次数：1] - [剩余数量：208] - [当前时间：14:43:48]

[ok] - 48822286 - 3
[尝试次数：1] - [剩余数量：207] - [当前时间：14:43:48]

[ok] - 48836080 - 1
[尝试次数：1] - [剩余数量：206] - [当前时间：14:43:48]

[ok] - 48836080 - 2
[尝试次数：1] - [剩余数量：205] - [当前时间：14:43:48]

[ok] - 48822286 - 4
[尝试次数：1] - [剩余数量：204] - [当前时间：14:43:48]

[ok] - 48824882 - 1
[尝试次数：1] - [剩余数量：203] - [当前时间：14:43:48]

[ok] - 48836080 - 3
[尝试次数：1] - [剩余数量：202] - [当前时间：14:43:48]

[ok] - 48822286 - 5
[尝试次数：1] - [剩余数量：201] - [当前时间：14:43:49]

[ok] - 48380427 - 1
[尝试次数：1] - [剩余数量：200] - [当前时间：14:43:49]

[ok] - 48836080 - 4
[尝试次数：1] - [剩余数量：199] - [当前时间：14:43:50]

[ok] - 48836114 - 1
[尝试次数：1] - [剩余数量：198] - [当前时间：14:43:50]

[ok] - 48840959 - 1
[尝试次数：1] - [剩余数量：197] - [当前时间：14:43:50]

[ok] - 48845525 - 1
[尝试次数：1] - [剩余数量：196] - [当前时间：14:43:51]

[ok] - 48082092 - 4
[尝试次数：2] - [剩余数量：195] - [当前时间：14:43:51]

[ok] - 48845535 - 1
[尝试次数：1] - [剩余数量：194] - [当前时间：14:43:51]

[ok] - 48836114 - 2
[尝试次数：1] - [剩余数量：193] - [当前时间：14:43:51]

[ok] - 48836114 - 3
[尝试次

[ok] - 49399196 - 28
[尝试次数：1] - [剩余数量：72] - [当前时间：14:44:17]

[ok] - 49399196 - 29
[尝试次数：3] - [剩余数量：71] - [当前时间：14:44:17]

[ok] - 49399196 - 30
[尝试次数：1] - [剩余数量：70] - [当前时间：14:44:17]

[ok] - 49399196 - 31
[尝试次数：1] - [剩余数量：69] - [当前时间：14:44:17]

[ok] - 49399196 - 32
[尝试次数：2] - [剩余数量：68] - [当前时间：14:44:18]

[ok] - 49399196 - 33
[尝试次数：1] - [剩余数量：67] - [当前时间：14:44:18]

[ok] - 49399196 - 34
[尝试次数：1] - [剩余数量：66] - [当前时间：14:44:18]

[ok] - 49399196 - 35
[尝试次数：1] - [剩余数量：65] - [当前时间：14:44:18]

[ok] - 49399196 - 36
[尝试次数：1] - [剩余数量：64] - [当前时间：14:44:19]

[ok] - 49399196 - 37
[尝试次数：1] - [剩余数量：63] - [当前时间：14:44:19]

[ok] - 49399196 - 38
[尝试次数：1] - [剩余数量：62] - [当前时间：14:44:19]

[ok] - 49399196 - 39
[尝试次数：1] - [剩余数量：61] - [当前时间：14:44:20]

[ok] - 49400880 - 1
[尝试次数：1] - [剩余数量：60] - [当前时间：14:44:20]

[ok] - 49399196 - 40
[尝试次数：1] - [剩余数量：59] - [当前时间：14:44:20]

[ok] - 49399196 - 41
[尝试次数：1] - [剩余数量：58] - [当前时间：14:44:20]

[ok] - 49400880 - 2
[尝试次数：1] - [剩余数量：57] - [当前时间：14:44:20]

[ok] - 49401028 - 1
[尝试次数：